
# Linear Programming Lab

## Total: 40 points

In this lab, you will explore a useful extension of linear programming duality (covered in class
LP Day 1) called linear programming sensitivity analysis. Here, we aim to find:

- How does the solution change when the objective function changes?
- How does the solution change when the resource limits change?
- How does the solution change when a constraint is added to the problem?

One approach to these types of questions is solving lots of slightly different linear programming problems. This method would work, but it is inelegant and (for large problems) computationally expensive (but in most cases today, computation is cheap and computing solutions to many problems is a standard technique to study sensitivity in practice). However, there is a more elegant solution
that gives these answers just by examining the dual information provided by the Simplex Algorithm.

**Consider the following scenario:** An instrument company makes trombones and trumpets. A trombone requires 40 square inches of metal sheet and a trumpet requires 30 square inches of metal. Metal costs $1 per square inch and 40,000 square inches are available. It takes two hours of skilled labor to make a novice quality trombone or a novice quality trumpet. Three more hours of skilled labor will turn a novice quality trombone into a professional trombone. Two more hours of skilled labor will turn a novice quality trumpet into a professional trumpet. There are 6000 hours of skilled labor available. Assume that you do not need to pay for labor. Assume that you can sell fractional trombones and trumpets. The market selling prices and production costs are given in the following table:

| Product    | Market Price | Cost (metal $1/sq.in.) |
| -------- | ------- | ------- |
| Novice Trombone | $70 | $40 |
| Professional Trombone | $140 | $40 |
| Novice Trumpet | $60 | $30 |
| Professional Trumpet | $110 | $30 |

---


# Problem 1: Formulate the LP (5 points)
## 1a: Set up the LP (3 points)

Write an objective function that maximizes profits (market price - production
costs) and all constraints.

### Instructions
- Write the equations, using LaTeX, between the $$ tags. Then convert the LP problem into code by referring to this documentation: [PuLP Guide](https://realpython.com/linear-programming-python/#using-pulp)

### Solution:
Let:
- $ x_1 $ be the number of novice trombones
- $ x_2 $ be the number of professional trombones
- $ x_3 $ be the number of novice trumpets
- $ x_4 $ be the number of professional trumpets

Objective Function: $$ Z = 30x_1 + 100x_2 + 30x_3 + 80x_4  $$
Constraints:
- Metal usage constraint: $$ 40(x_1 + x_2) + 30(x_3 + x_4) \leq 40,000 $$
- Labor usage constraint: $$ 2(x_1 + x_3) + 5x_2 + 4x_4 \leq 6000 $$
- Non-negativity constraint: $$ x_1, x_2, x_3, x_4 \geq 0 $$

In [78]:
from pulp import LpMaximize, LpProblem, LpVariable, LpStatus, PULP_CBC_CMD
import copy

# Define the problem
main_model = LpProblem(name="instrument-production", sense=LpMaximize) # (please do not rename this variable as it's used later in the notebook)

# Define variables
x1 = LpVariable(name="x1", lowBound=0)  # Novice Trombone
x2 = LpVariable(name="x2", lowBound=0)  # Professional Trombone
x3 = LpVariable(name="x3", lowBound=0)  # Novice Trumpet
x4 = LpVariable(name="x4", lowBound=0)  # Professional Trumpet

# Objective function
# TODO: Add objective function
main_model += 30*x1 + 100*x2 + 30*x3 + 80*x4

# Constraints
# 1. Total metal sheet used
# TODO: Add constraint
main_model += (40 * (x1 + x2) + 30 * (x3 + x4)) <= 40000, "Metal_Sheet_Constraint"

# 2. Total skilled labor hours used
# TODO: Add constraint
main_model += (2*(x1+x3) + 5*x2+4*x4) <= 6000, "Skilled_Labor_Constraint"

# 3. Non-negativity constraints (already enforced by lowBound=0 in variable definition)
main_model += x1 >= 0
main_model += x2 >= 0
main_model += x3 >= 0
main_model += x4 >= 0

## 1b) Solve the LP Computationally (1 point)
Use the model we created above to find the optimal profit. Run the code block below to solve the problem.

### Solution:

Optimal Profit: $106,666.664

In [79]:
# Solve the problem
main_model.solve()
main_model.objective.value()

106666.664

## 1c.) What is the optimal number of each instrument to produce? (1 point)
Run the code cell below to find the optimal value for each variable in the model. Fill in the table below.
### Solution:

| Variable    | Value |
| -------- | ------- |
| x1 | 0 |
| x2 | 0 |
| x3 | 0 |
| x4 | 1333.333 |

In [80]:
for var in main_model.variables():
    print(var.name, var.value())

x1 0.0
x2 0.0
x3 0.0
x4 1333.3333


# Problem 2: Sensitivity of Novice Quality Trumpets (5 points)

## 2a) (2 points)

What would happen if the market price of novice quality trumpets went up? In other words, how much can the market price coefficient in the objective function change without causing the optimal basis to change? This is also called “allowable increase.” You are provided a helper function to find the allowable increase. Try messing with the parameters and looking at the source code in the `helpers.py` file to understand how it works.

### Solution:

Because the limiting variable is the quantity of metal, the solution is to make only professional trumpets, which nets the most profit per metal use, despite requiring more time. Thus, the price of novice trumpets has to increase to greater than the price of professional trumpets to change the optimal basis.

In [81]:
from helpers import find_allowable_increase
# Run the helper to find the allowable increase for x3
allowable_increase = find_allowable_increase(base_model=main_model, var_name="x3", base_coef=30, step=5)

print(f"The allowable increase for x3 is: {allowable_increase}")

Iteration 1: Coefficient of x3: 30, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 2: Coefficient of x3: 35, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 3: Coefficient of x3: 40, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 4: Coefficient of x3: 45, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 5: Coefficient of x3: 50, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 6: Coefficient of x3: 55, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
---------------------------------------

C:\Users\ilum\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pulp\pulp.py:1650: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


Iteration 7: Coefficient of x3: 60, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 8: Coefficient of x3: 65, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 9: Coefficient of x3: 70, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 10: Coefficient of x3: 75, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 11: Coefficient of x3: 80, Objective value: 106666.664, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 0.0, 'x4': 1333.3333}
----------------------------------------
Iteration 12: Coefficient of x3: 85, Objective value: 113333.3305, Solution: {'x1': 0.0, 'x2': 0.0, 'x3': 1333.3333, 'x4': 0.0}
-----------------------------------

## 2b) (2 points)
Would novice trumpets be more profitable than professional trumpets to produce if novice trumpets sold for the same amount as professional quality trumpets?

### Solution:
No, the profit would be the same, but they would save somec labor hours.

## 2c) (1 point)
What would change if the price went down?

### Solution:
Given that metal is the limiting variable, the only thing that matters is the ratio of price to metal. It is always optimal to make the product that gets the most profit per amount of metal.

# Problem 3: Sensitivity of Novice Quality Trombones (5 points)

## 3a) (2 points)

What would happen if the market price of novice quality trombones went up? In other words, what is the “allowable increase” in the market price of novice quality trombones?


### Solution:
The allowable increase is 77. Once the price of novice trombones increases to 147 (profit of 107), then it becomes optimal to just make novice trombones.

In [82]:
from helpers import find_allowable_increase

# setting verbose to False since we take smaller steps for this variable and don't want to print every iteration
allowable_increase = find_allowable_increase(base_model=main_model, var_name="x1", base_coef=60, step=1, max_iter=100, verbose=False)

print(f"The allowable increase for x1 is: {allowable_increase}")

Optimal basis changed at coefficient 137.

The allowable increase for x1 is: 77


## 3b) (3 points)
Would they be profitable to produce if they sold for the same amount as professional quality trombones? What would make them profitable to produce (intuitive explanation is expected, not specific numbers)?

### Solution:

If the price of novice trombones is 147, then it would not be more profitable to sell professional trombones until the price of professional trombones is increased to greater than 147.

# Problem 4: Sensitivity of Professional Quality Trumpets (2 points)

What would happen to the optimal solution if the price of professional quality trumpets fell to $100? If there is a change, what would the new production plan and optimal profit be?

Provide your analysis and corresponding code below.

### Solution:

If the price of professional trumpets falls to 100, the it is optimal to just make professional trombones.

In [83]:
low_trumpet_model = LpProblem(name="instrument-production", sense=LpMaximize)

# Define variables
x1 = LpVariable(name="x1", lowBound=0)  # Novice Trombone
x2 = LpVariable(name="x2", lowBound=0)  # Professional Trombone
x3 = LpVariable(name="x3", lowBound=0)  # Novice Trumpet
x4 = LpVariable(name="x4", lowBound=0)  # Professional Trumpet

# TODO: Re-define Objective function
low_trumpet_model += 30 * x1 + 100 * x2 + 30 * x3 + 70 * x4

# Constraints
low_trumpet_model += 40 * x1 + 40 * x2 + 30 * x3 + 30 * x4 <= 40000, "Metal Sheet Constraint"
low_trumpet_model += 2 * x1 + 5 * x2 + 2 * x3 + 4 * x4 <= 6000, "Skilled Labor Constraint"

# Solve the low_trumpet_model
low_trumpet_model.solve(PULP_CBC_CMD(msg=0))

# Display results
print("Sensitivity Analysis for Professional Quality Trumpets (Price = $100):")
print(f"Objective value (Profit): {low_trumpet_model.objective.value()}")
for var in low_trumpet_model.variables():
    print(f"{var.name}: {var.value()}")

Sensitivity Analysis for Professional Quality Trumpets (Price = $100):
Objective value (Profit): 100000.0
x1: 0.0
x2: 1000.0
x3: 0.0
x4: 0.0


# Problem 5: Sensitivity of Metal Resources (8 points)

## 5a) (4 points)
How do production plans change when metal supplies change? At which point does metal stop being a limiting resource and how do you know?

**HINT**: To find out how the production plans change when metal supplies change, you can use a **guess-and-check approach**. 

1. Start with the current RHS of the "Metal Sheet Constraint" (40,000 square inches).
2. Gradually increase or decrease the RHS (e.g., by 1,000 or 5,000 units at a time).
3. For each adjustment:
   - Re-run to solve the problem and record the production plan (values of x_1, x_2, x_3, x_4) and the total profit.
   - Compare the new solution to the original solution (at RHS = 40,000).
4. Stop increasing when you notice the production plan has stopped changing. The last RHS value before the variable values stop changing is the **RHS Tolerance Upper Bound**.
5. Stop decreasing when the model becomes infeasible. The last RHS value before the model becomes infeasible is the **RHS Tolerance Lower Bound**.

### Solution:

Metal stops being the limiting resource at 45,000. This can also be calculated as with 6000 labor hours, you can only make 1500 professional trumpets, which takes 45,000 metal. Which more metal, profressional trumpets and trombones make the same amount of money per labor hour.

In [84]:
rhs_tol_model = copy.deepcopy(main_model)

# Print the results
print("Initial Solution:")
print(f"Total Profit: ${rhs_tol_model.objective.value():.2f}")
print("Production Plan:")
for var in rhs_tol_model.variables():
    print(f"  {var.name}: {var.varValue}")

# Check slack for each constraint
print("\nSlack Values for Constraints:")
for name, constraint in rhs_tol_model.constraints.items():
    print(f"  {name}: RHS = {constraint.constant}, slack = {constraint.slack}")

# ---------------------------------------------
# Guess-and-Check Setup
# ---------------------------------------------
# Change the RHS of the "Metal Sheet Constraint" below and re-run this block of code.
rhs_tol_model.constraints["Metal_Sheet_Constraint"].changeRHS(45000)  # TODO: Change this value

print("---------------------------------------------")
print("After Adjusting Metal Sheet Constraint:")

# Re-solve the problem
rhs_tol_model.solve(PULP_CBC_CMD(msg=0))
print(f"Model Solution Status: {LpStatus[rhs_tol_model.status]}")

# Print updated results
print(f"Total Profit: ${rhs_tol_model.objective.value():.2f}")
print("Production Plan:")
for var in rhs_tol_model.variables():
    print(f"  {var.name}: {var.varValue}")

# Check slack for each constraint
print("\nSlack Values for Constraints:")
for name, constraint in rhs_tol_model.constraints.items():
    print(f"  {name}: RHS = {constraint.constant}, slack = {constraint.slack}")

Initial Solution:
Total Profit: $106666.66
Production Plan:
  x1: 0.0
  x2: 0.0
  x3: 0.0
  x4: 1333.3333

Slack Values for Constraints:
  Metal_Sheet_Constraint: RHS = -40000, slack = -0.0
  Skilled_Labor_Constraint: RHS = -6000, slack = 666.6666999999998
  _C1: RHS = 0, slack = -0.0
  _C2: RHS = 0, slack = -0.0
  _C3: RHS = 0, slack = -0.0
  _C4: RHS = 0, slack = -1333.3333
---------------------------------------------
After Adjusting Metal Sheet Constraint:
Model Solution Status: Optimal
Total Profit: $120000.00
Production Plan:
  x1: 0.0
  x2: 0.0
  x3: 0.0
  x4: 1500.0

Slack Values for Constraints:
  Metal_Sheet_Constraint: RHS = -45000, slack = -0.0
  Skilled_Labor_Constraint: RHS = -6000, slack = -0.0
  _C1: RHS = 0, slack = -0.0
  _C2: RHS = 0, slack = -0.0
  _C3: RHS = 0, slack = -0.0
  _C4: RHS = 0, slack = -1500.0


## 5b) (4 points)
**What is the shadow price of metal?**  
1. Explain the concept of a shadow price in the context of the "Metal Sheet Constraint" and how it impacts the objective function (profit).  
2. Using the shadow price, calculate how much the profit is expected to change if the metal supplies increase by 500 square inches.  
3. Solve the problem again with the increased metal supplies and verify the expected profit change and the new production plan.

Use the following formula for profit change:  
$$ \Delta profit = \Delta \text{metal supplies quantity} \cdot dual_{metal} $$

---

### HINT

1. **What is the Shadow Price?**  
   The shadow price (also called the dual value) of a constraint tells you how much the objective value (profit) will change per unit increase in the RHS of the constraint, as long as the change is within the **RHS tolerance interval**.

2. **How to Find the Shadow Price?**  
   - Solve the LP model with the original RHS.
   - Check the **dual value** (`constraint.pi`) of the "Metal Sheet Constraint" in the solver's output. This is the shadow price.

3. **Useful Methods**
   - Use `shadow_model.constraints["Metal_Sheet_Constraint"].changeRHS(new_rhs)` to update the RHS of the "Metal Sheet Constraint"

### Solution:

The shadow price of the metal is 2.667, or 80/30. This makes sense as the profit professional trumpets is $80, and they take 30 metal to make. We expect the profit to increase by 1333.333 when we get 500 more metal, and we see that it increases from 106666.664 to 108000.

In [85]:
shadow_model = copy.deepcopy(main_model)

# TODO: Solve the shadow price problem
print("If metal supply increases by 500, we expect to see increase in profit of $", shadow_model.constraints["Metal_Sheet_Constraint"].pi * 500)

# TODO: Update RHS
shadow_model.constraints["Metal_Sheet_Constraint"].changeRHS(40500)
shadow_model.solve()

# Display the new results
print("New Results with Increased Metal Supply:")
print(f"New Profit: {shadow_model.objective.value()}")
print("Production Plan:")
for var in shadow_model.variables():
    print(f"  {var.name}: {var.value()}")

If metal supply increases by 500, we expect to see increase in profit of $ 1333.3333499999999
New Results with Increased Metal Supply:
New Profit: 108000.0
Production Plan:
  x1: 0.0
  x2: 0.0
  x3: 0.0
  x4: 1350.0


# Problem 6: Sensitivity of Labor Resources (5 points)

## 6a) (1 point)
How much labor is unused?

HINT: What is the slack?

666.667 hours of labor unused

In [86]:
# Check slack for each constraint
print("\nSlack Values for Constraints:")
for name, constraint in main_model.constraints.items():
    print(f"  {name}: RHS = {constraint.constant}, slack = {constraint.slack}")


Slack Values for Constraints:
  Metal_Sheet_Constraint: RHS = -40000, slack = -0.0
  Skilled_Labor_Constraint: RHS = -6000, slack = 666.6666999999998
  _C1: RHS = 0, slack = -0.0
  _C2: RHS = 0, slack = -0.0
  _C3: RHS = 0, slack = -0.0
  _C4: RHS = 0, slack = -1333.3333


## 6b) (2 points)
How much would you be willing to pay an additional laborer?

### Solution:

In the current state, none, as there is already excess labor.

## 6c) (2 points)
Suppose that industrial regulation complicate the process so it takes an additional hour of labor to turn a novice quality instrument into a professional quality instrument. How and why does that change affect the optimal production plan and profit?

### Solution:

This change makes it optimal to just make professional trombones, which actually perfectly uses all metal and labor hours.

In [89]:
more_labor_model = LpProblem(name="instrument-production", sense=LpMaximize)

# Define variables
x1 = LpVariable(name="x1", lowBound=0)  # Novice Trombone
x2 = LpVariable(name="x2", lowBound=0)  # Professional Trombone
x3 = LpVariable(name="x3", lowBound=0)  # Novice Trumpet
x4 = LpVariable(name="x4", lowBound=0)  # Professional Trumpet

# TODO: Re-define Objective function
more_labor_model += 30 * x1 + 100 * x2 + 30 * x3 + 80 * x4

# Constraints
more_labor_model += (
    40 * x1 + 40 * x2 + 30 * x3 + 30 * x4 <= 40000,
    "Metal Sheet Constraint",
)
more_labor_model += (
    2 * x1 + 6 * x2 + 2 * x3 + 5 * x4 <= 6000,
    "Skilled Labor Constraint",
)

# Solve the more_labor_model
more_labor_model.solve(PULP_CBC_CMD(msg=0))

# Display results
print(f"Objective value (Profit): {more_labor_model.objective.value()}")
for var in more_labor_model.variables():
    print(f"{var.name}: {var.value()}")

print("\nSlack Values for Constraints:")
for name, constraint in more_labor_model.constraints.items():
    print(f"  {name}: RHS = {constraint.constant}, slack = {constraint.slack}")

Objective value (Profit): 100000.0
x1: 0.0
x2: 1000.0
x3: 0.0
x4: 0.0

Slack Values for Constraints:
  Metal_Sheet_Constraint: RHS = -40000, slack = -0.0
  Skilled_Labor_Constraint: RHS = -6000, slack = -0.0


# Problem 7: Sensitivity of New Constraint (5 points)

## 7a) (4 points)
The owner of the company comes up with a design for a beautiful hand-crafted tuba. Each tuba requires 250 hours of labor (6.25 weeks of full time work) and uses 50 square inches of metal. If the
company can sell tubas for $200, would it be worthwhile to produce? Use:
$$ \text{(marginal cost of metal)}_{tuba} = \text{(metal used)}_{tuba} \cdot dual_{metal} $$

Discuss how looking at the shadow prices of metal and labor can help
you intuit whether diverting metal resources to tuba production is a good idea. Check your intuition with the solver.

### Solution:

Tubas give the company a more money efficient way to use metal, but the issue then becomes labor. The company makes tubas until the remaining metal and labor lines up to use all of it by just making professional trumpets.

The shadow price of metal actually goes down a bit to 2.658 because metal is no longer the sole limiter and the company doesn't have an excess of labor. The shadow price of labor goes to 0.068 (previously 0) as extra labor can now actually be used to make more tubas.

In [108]:
from pulp import LpMaximize, LpProblem, LpVariable, value

# Define the problem
tuba_model = LpProblem(name="instrument-production", sense=LpMaximize)

# Define variables
x1 = LpVariable(name="x1", lowBound=0)  # Novice Trombone
x2 = LpVariable(name="x2", lowBound=0)  # Professional Trombone
x3 = LpVariable(name="x3", lowBound=0)  # Novice Trumpet
x4 = LpVariable(name="x4", lowBound=0)  # Professional Trumpet
x5 = LpVariable(name="x5", lowBound=0)  # Tubas

# TODO: Define the problem and analyze the shadow prices
tuba_model += 30 * x1 + 100 * x2 + 30 * x3 + 80 * x4 + 150 * x5

tuba_model += 40 * x1 + 40 * x2 + 30 * x3 + 30 * x4 + 50 * x5 <= 40000, "Metal_Sheet_Constraint"
tuba_model += 2 * x1 + 5 * x2 + 2 * x3 + 4 * x4 + 250 * x5 <= 6000, "Skilled_Labor_Constraint"

# Solve
tuba_model.solve(PULP_CBC_CMD(msg=0))

# Display results
print(f"Objective value (Profit): {tuba_model.objective.value()}")
for var in tuba_model.variables():
    print(f"{var.name}: {var.value()}")

print("\nSlack Values for Constraints:")
for name, constraint in tuba_model.constraints.items():
    print(f"  {name}: RHS = {constraint.constant}, slack = {constraint.slack}")

Objective value (Profit): 106712.3269
x1: 0.0
x2: 0.0
x3: 0.0
x4: 1328.7671
x5: 2.739726

Slack Values for Constraints:
  Metal_Sheet_Constraint: RHS = -40000, slack = -0.0
  Skilled_Labor_Constraint: RHS = -6000, slack = -0.0


In [104]:
print(tuba_model.constraints["Metal_Sheet_Constraint"].pi)
print(tuba_model.constraints["Skilled_Labor_Constraint"].pi)

2.6575342
0.068493151


## 7b) (1 point)
Is it profitable if tubas sell for $150? Why?

### Solution:

It is still technically profitable, as they would sell for $100 more than the material cost, but it is not optimal when factoring in other options and limited materials. This is becuase the profit per unit of metal drops to 2:
$$ \frac{\text{selling price} - \text{cost of metal}}{\text{metal used}} = \frac{150 - 50}{50} = 2 $$
which is much less than the profit per unit of metal for professional trumpets.

# Problem 8: Making Sense of the Dual Problem (5 points)

## 8a) (3 points)
Set up the original problem’s corresponding dual LP. Write an objective function that maximizes profits (market price - production costs) and all dual constraints.

In [110]:
from pulp import LpMinimize, LpProblem, LpVariable

# Define the dual problem
dual_model = LpProblem(name="dual-instrument-production", sense=LpMinimize)

# TODO: Define the dual problem
y1 = LpVariable(name="y1", lowBound=0) # price of metal
y2 = LpVariable(name="y2", lowBound=0) # price of labor

# minimize cost of materials
dual_model += 40000 * y1 + 6000 * y2

# the resources can't be less than the profit of the item (supplier's perspective)
dual_model += 40 * y1 + 2 * y2 >= 30
dual_model += 40 * y1 + 5 * y2 >= 100
dual_model += 30 * y1 + 2 * y2 >= 30
dual_model += 30 * y1 + 4 * y2 >= 80

dual_model.solve()

# Display results
print(f"Objective value (Profit): {dual_model.objective.value()}")
for var in dual_model.variables():
    print(f"{var.name}: {var.value()}")

print("\nSlack Values for Constraints:")
for name, constraint in dual_model.constraints.items():
    print(f"  {name}: RHS = {constraint.constant}, slack = {constraint.slack}")

Objective value (Profit): 106666.66799999999
y1: 2.6666667
y2: 0.0

Slack Values for Constraints:
  _C1: RHS = -30, slack = -76.66667
  _C2: RHS = -100, slack = -6.666669999999996
  _C3: RHS = -30, slack = -50.0
  _C4: RHS = -80, slack = -0.0


## 8b) (1 point)
Run the code block below to solve the problem. What is the optimal profit? Give a plot of the constraints (you can use this [example](https://github.com/hakeemrehman/Python-PuLP-/blob/master/LP%20Using%20PuLP%20and%20Graphical%20Sol..ipynb) for guidance).

In [111]:
# Solve the dual problem
dual_model.solve(PULP_CBC_CMD(msg=0))

# Display the results
print("Dual Problem Results:")
print(f"Optimal Value (Total Resource Value): {dual_model.objective.value()}")

Dual Problem Results:
Optimal Value (Total Resource Value): 106666.66799999999


## 8c) (1 point)
Say something about the relationship between dual variables and slackness in the primal and dual problem.

### Solution:

The dual variables are the prices of metal and labor. The slack in the dual problem represents how much more an intstrument would need to sell for for it to be optimal to produce it.

In the primal, the variables are the number of each instrument produced. The slack in the primal presents how much excess there is of each resource.